In [ ]:
# Instalando dependências básicas
!pip install pandas streamlit requests python-dotenv

# Instalando bibliotecas para o LLM Open Source (Usaremos Transformers para carregar um modelo)
!pip install transformers accelerate bitsandbytes sentencepiece


Passo A: No SEU COMPUTADOR (Onde a API está rodando)
Instale o ngrok:

Baixe o ngrok no site oficial e adicione-o ao PATH ou use-o na pasta onde ele foi baixado.

Crie uma conta gratuita no ngrok e obtenha seu Auth Token.

Exponha sua API (a porta 3000): Abra seu terminal ou prompt de comando (no seu computador) e execute:

ngrok http 3000

Anote o URL Público: O ngrok irá gerar um URL público temporário (ex: https://abcd-123-45-678-90.ngrok-free.app). Este é o seu novo URL_API.


In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import display, Markdown
import os
import requests

# --- CONFIGURAÇÃO DE ACESSO À API (SUBSTITUA PELOS SEUS VALORES REAIS) ---
NGROK_URL = "SUA_URL_NGROK_AQUI"
URL_API_REAL = f"{NGROK_URL}/api/transactions/dashboard"
API_DASHBOARD_KEY_VALUE = "SUA_CHAVE_API_AQUI"
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

# --- FUNÇÕES DE CARREGAMENTO E PROCESSAMENTO DE DADOS ---

def map_region(estado):
    if not estado: return "Desconhecida"
    estado = estado.upper()
    if estado in ["AC", "AP", "AM", "PA", "RO", "RR", "TO"]: return "Norte"
    elif estado in ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"]: return "Nordeste"
    elif estado in ["DF", "GO", "MT", "MS"]: return "Centro-Oeste"
    elif estado in ["ES", "MG", "RJ", "SP"]: return "Sudeste"
    elif estado in ["PR", "RS", "SC"]: return "Sul"
    else: return "Desconhecida"

def carregar_dados_api_real():
    headers = {"x-api-key": API_DASHBOARD_KEY_VALUE}
    print(f"Tentando conectar na API: {URL_API_REAL}")
    try:
        response = requests.get(URL_API_REAL, headers=headers, timeout=15)
        response.raise_for_status()

        df_bruto = pd.DataFrame(response.json().get("data", []))
        if df_bruto.empty: return pd.DataFrame()

        # Flatten
        if 'customer' in df_bruto.columns and 'product' in df_bruto.columns:
            df_bruto = pd.concat([
                df_bruto.drop(['customer', 'product'], axis=1),
                df_bruto['customer'].apply(pd.Series).add_prefix('customer_'),
                df_bruto['product'].apply(pd.Series).add_prefix('product_')
            ], axis=1)

        # Renomear e processar
        df_renomeado = df_bruto.rename(columns={
            'totalPrice': 'preco_total_item', 'date': 'data_pedido', 'status': 'status_pedido',
            'customer_name': 'nome_cliente', 'customer_segment': 'segmento_cliente',
            'customer_city': 'cidade', 'customer_state': 'estado',
            'product_name': 'nome_produto', 'product_quantity': 'quantidade'
        })

        df_renomeado['regiao'] = df_renomeado['estado'].apply(map_region)
        df_renomeado['data_pedido'] = pd.to_datetime(df_renomeado['data_pedido'])
        df_renomeado['mes_ano'] = df_renomeado['data_pedido'].dt.to_period('M').astype(str)
        return df_renomeado

    except requests.exceptions.RequestException as e:
        print(f"ERRO DE CONEXÃO: {e}. Usando dados simulados como fallback.")
        return pd.DataFrame()

# --- FUNÇÃO DE FALLBACK PARA DADOS SIMULADOS ---
def gerar_dados_simulados():
    np.random.seed(42)
    num_pedidos = 1000
    estados = ["SP", "RJ", "MG", "BA", "PE", "RS", "PR", "AM", "GO"]
    pesos_produtos = np.linspace(0.01, 0.1, 20)
    probabilidades_produtos = pesos_produtos / np.sum(pesos_produtos)
    data = {
        'orderId': np.arange(1000, 1000 + num_pedidos), 'preco_total_item': np.round(np.random.uniform(10.0, 500.0, num_pedidos), 2),
        'data_pedido': pd.to_datetime(pd.date_range(start='2024-01-01', periods=num_pedidos, freq='D')),
        'status_pedido': np.random.choice(["Completed", "Pending", "Canceled", "Processing"], num_pedidos, p=[0.75, 0.15, 0.05, 0.05]),
        'nome_cliente': [f"Cliente_{i % 100}" for i in range(num_pedidos)],
        'segmento_cliente': np.random.choice(["Gold", "Silver", "Bronze", "Standard"], num_pedidos, p=[0.15, 0.25, 0.35, 0.25]),
        'cidade': [f"Cidade_{i % 50}" for i in range(num_pedidos)], 'estado': np.random.choice(estados, num_pedidos),
        'nome_produto': np.random.choice([f"Produto_{i}" for i in range(20)], num_pedidos, p=probabilidades_produtos),
        'quantidade': np.random.randint(1, 6, num_pedidos)
    }
    df = pd.DataFrame(data)
    df['regiao'] = df['estado'].apply(map_region)
    df['mes_ano'] = df['data_pedido'].dt.to_period('M').astype(str)
    return df

# --- CARREGAMENTO DE DADOS E KPIS ---
df = carregar_dados_api_real()

if df.empty:
    df = gerar_dados_simulados()

receita_total = df['preco_total_item'].sum()
pedidos_unicos = df['orderId'].nunique()
df_kpi = pd.DataFrame([{'receita_total': receita_total, 'pedidos_unicos': pedidos_unicos, 'ticket_medio': receita_total / pedidos_unicos if pedidos_unicos > 0 else 0}])
print(f"✅ Análise pronta para {len(df)} registros.")

# --- AGENTE DE IA (CARREGAMENTO DO MODELO) ---
try:
    global tokenizer, model
    _ = tokenizer; _ = model
    print(f"Modelo {MODEL_NAME} já carregado. Pulando carregamento.")
except NameError:
    print(f"Carregando Modelo {MODEL_NAME} (Phi-3)...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True
        )
        print(f"Modelo {MODEL_NAME} carregado com sucesso.")
    except Exception as e:
        print(f"ERRO FATAL ao carregar o modelo: {e}")
        raise SystemExit(e)

# --- FUNÇÃO DE INFERÊNCIA ---
def generate_insights(prompt):
    messages = [{"role": "user", "content": prompt}]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(model.device)

    generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(generated_ids[0], skip_special_tokens=False)

    # Limpeza da resposta do Phi-3
    try:
        start_index = response.index('[/INST]') + len('[/INST]')
        return response[start_index:].strip()
    except ValueError:
        return response.strip()

# --- PREPARAÇÃO DOS DADOS E PROMPT ---
data_summary = f"""
## Resumo do Dataframe de Vendas:
* **Total de Pedidos:** {df_kpi['pedidos_unicos'].iloc[0]}
* **Receita Total:** R$ {df_kpi['receita_total'].iloc[0]:,.2f}
* **Ticket Médio Global:** R$ {df_kpi['ticket_medio'].iloc[0]:,.2f}

## Agregados Relevantes:
* **Receita por Status:**
{df.groupby('status_pedido')['preco_total_item'].sum().sort_values(ascending=False).to_markdown()}
* **Receita por Região (Top 3):**
{df.groupby('regiao')['preco_total_item'].sum().nlargest(3).to_markdown()}
* **Produtos Mais Vendidos (Top 5 em Receita):**
{df.groupby('nome_produto')['preco_total_item'].sum().nlargest(5).to_markdown()}
* **Segmento de Cliente (Receita Total):**
{df.groupby('segmento_cliente')['preco_total_item'].sum().sort_values(ascending=False).to_markdown()}
"""

analyst_prompt = f"""
Você é um Analista de Dados Sênior, fluente em Português do Brasil.
Sua tarefa é analisar os dados de vendas fornecidos e produzir uma análise focada em Tomada de Decisão.

Instruções Cruciais:
1.  **IDIOMA:** A resposta DEVE ser 100% em Português do Brasil.
2.  **FORMATO:** Gere 5 insights essenciais, formatados como uma lista numerada.
3.  **ESTRUTURA:** Cada ponto deve seguir esta estrutura EXATA, utilizando negrito:
    * **Insight Chave:** [O fato principal e a evidência numérica (R$, o símbolo oficial do Real, a moeda brasileira) extraída dos dados.]
    * **Recomendação Acionável:** [O que a equipe de vendas/marketing deve fazer com base no insight.]

4.  **REGRAS RÍGIDAS DE FORMATAÇÃO:**
    * NÃO use as palavras 'Analyze', 'Analysis', 'Recommendation' ou 'Recomendação' como cabeçalhos.
    * Use o formato monetário (R$) corretamente.

--- DADOS PARA ANÁLISE ---
{data_summary}
--- FIM DOS DADOS ---

## 📊 Análise de 5 Pontos com Ações Recomendadas:
"""

# --- EXECUÇÃO E EXIBIÇÃO ---
print("\n" + "="*50)
print("🤖 AGENTE DE INSIGHTS DE IA EM EXECUÇÃO...")
print("="*50)

insights = generate_insights(analyst_prompt)

print("\n" + "#"*50)
display(Markdown("## ✨ Insights Gerados pelo Modelo Phi-3 Mini"))
display(Markdown(insights))
print("#"*50)